# Conditional

`gtsam.Conditional` is the base class for conditional probability distributions or densities that result from variable elimination. Conditionals are essentially specialized factors representing $P(	ext{Frontals} | 	ext{Parents})$.

Like `gtsam.Factor`, you typically don't instantiate `gtsam.Conditional` directly. Instead, you work with derived classes obtained from elimination, such as:
*   `gtsam.GaussianConditional`
*   `gtsam.DiscreteConditional`
*   `gtsam.HybridGaussianConditional`
*   `gtsam.SymbolicConditional`

This notebook demonstrates the common interface provided by the base class.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/inference/doc/Conditional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

In [ ]:
import gtsam
import numpy as np

# We need concrete graph types and elimination to get a Conditional
from gtsam import GaussianFactorGraph, Ordering
from gtsam import symbol_shorthand

X = symbol_shorthand.X
L = symbol_shorthand.L

## Example: Obtaining and Inspecting a Conditional

We'll create a simple `GaussianFactorGraph` and eliminate one variable to get a `GaussianConditional`.

In [ ]:
# Create a simple Gaussian Factor Graph P(x0) P(x1|x0)
graph = GaussianFactorGraph()
model1 = gtsam.noiseModel.Isotropic.Sigma(1, 1.0)
model2 = gtsam.noiseModel.Isotropic.Sigma(1, 1.0)

# Prior on x0
graph.add(X(0), -np.eye(1), np.zeros(1), model1)
# Factor between x0 and x1
graph.add(X(0), -np.eye(1), X(1), np.eye(1), np.zeros(1), model2)

print("Eliminating x0 from graph:")
graph.print()

# Eliminate x0
ordering = Ordering([X(0)])
bayes_net, remaining_graph = graph.eliminatePartialSequential(ordering)

print("\nResulting BayesNet:")
bayes_net.print()

# Get the resulting conditional P(x0 | x1)
# In this case, it's a GaussianConditional
conditional = bayes_net.at(0) # or bayes_net[0]

# Access methods from the Conditional base class
print(f"Conditional Keys (all): {conditional.keys()}")
print(f"Number of Frontals: {conditional.nrFrontals()}")
print(f"Frontal Keys: {conditional.frontals()} ({gtsam.DefaultKeyFormatter(list(conditional.frontals())[0])})")
print(f"First Frontal Key: {conditional.firstFrontalKey()} ({gtsam.DefaultKeyFormatter(conditional.firstFrontalKey())})")
print(f"Number of Parents: {conditional.nrParents()}")
print(f"Parent Keys: {conditional.parents()} ({gtsam.DefaultKeyFormatter(list(conditional.parents())[0])})")

# Conditional objects can also be printed
# conditional.print("P(x0 | x1): ")

Eliminating x0 from graph:
Factor Graph: size 2
Factor 0: JacobianFactor(keys = [8070450532247928832], Z = [ -1 ], b = [ 0 ], model = diagonal sigmas [1])
Factor 1: JacobianFactor(keys = [8070450532247928832; 8070450532247928833], A[0] = [ -1  1 ], b = [ 0 ], model = diagonal sigmas [1])


Resulting BayesNet: size 1
Conditional 0: GaussianConditional( P(x0 | x1) = dx0 - R*dx1 - d), R = [ 0.5 ], d = [ 0 ], sigmas = [ 0.707107 ])


Conditional Keys (all): [8070450532247928832, 8070450532247928833]
Number of Frontals: 1
Frontal Keys: [8070450532247928832] (x0)
First Frontal Key: 8070450532247928832 (x0)
Number of Parents: 1
Parent Keys: [8070450532247928833] (x1)


## Evaluation (Derived Class Methods)

Concrete conditional classes provide methods like `logProbability(values)` or `evaluate(values)` to compute the conditional probability (or density) given values for the parent variables. These methods are defined in the derived classes, not the `Conditional` base class itself.

In [ ]:
# Example for GaussianConditional (requires VectorValues)
vector_values = gtsam.VectorValues()
# vector_values.insert(X(0), np.array([0.0])) # Value for frontal variable
vector_values.insert(X(1), np.array([1.0])) # Value for parent variable

# These methods are specific to GaussianConditional / other concrete types
try:
    log_prob = conditional.logProbability(vector_values)
    # print(f"\nLog Probability P(x0|x1=1.0): {log_prob}")
    prob = conditional.evaluate(vector_values)
    # print(f"Probability P(x0|x1=1.0): {prob}")
except AttributeError:
    print("\nNote: logProbability/evaluate called on base Conditional pointer, needs derived type.")
    # In C++, you'd typically have a shared_ptr<GaussianConditional>.
    # In Python, if you know the type, you might access methods directly,
    # but the base class wrapper doesn't expose derived methods.
    pass

# To properly evaluate, you often use the BayesNet/BayesTree directly
# bayes_net.logProbability(vector_values)